# `nanoGPT`: GPT-2 Small (150M Params)

## Install / Setup

### Google Colab

1. Mount Google Drive

    ```python
    from google.colab import drive
    drive.mount('/content/drive')
    ```

2. `bfloat16` doesn't work on Colab, switch to `float16`

3. To prepare data
    ```bash
    !python3 drive/MyDrive/Google-Colab/nanoGPT/data/prepare.py yelp_review_full
    ```

### First Time Running

We need to install `ngpt` and setup the Shakespeare dataset

This will need to be ran the first time you are running this notebook.

Following the

```python
!python3 -m pip install nanoGPT
```

you will need to restart your runtime (Runtime -> Restart runtime)

After this, you should be able to

```python
>>> import ngpt
>>> ngpt.__file__
'/content/nanoGPT/src/ngpt/__init__.py'
```

In [1]:
%%bash

#cd drive/MyDrive/Google-Colab/
echo "pwd: $(pwd)"

HF_DATASETS_CACHE="./.cache/huggingface"
mkdir -p "${HF_DATASETS_CACHE}"

python3 -c 'import ngpt; print(ngpt.__file__)' 2> '/dev/null'
STATUS=$?

if [[ $STATUS -eq 0 ]]; then
    echo "Has ngpt installed. Nothing to do."
else
    echo "Does not have ngpt installed. Installing..."
    git clone 'https://github.com/saforem2/nanoGPT'
    python3 -m pip install -e nanoGPT
fi

#python3 nanoGPT/data/prepare.py yelp_review_full

#cd -

pwd: /lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/notebooks
/lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/src/ngpt/__init__.py
Has ngpt installed. Nothing to do.


## Post Install

If installed correctly, you should be able to:

```python
>>> import ngpt
>>> ngpt.__file__
'/path/to/nanoGPT/src/ngpt/__init__.py'
```

In [2]:
%load_ext autoreload
%autoreload 2

import ngpt
from enrich import get_logger
log = get_logger('jupyter')
log.info(ngpt.__file__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[2023-11-30 07:10:16][INFO][3434626787.py:7] - /lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/src/ngpt/__init__.py


## Build Trainer

Explicitly, we:

1. `setup_torch(...)`
2. Build `cfg: DictConfig = get_config(...)`
3. Instnatiate `config: ExperimentConfig = instantiate(cfg)`
4. Build `trainer = Trainer(config)`

In [3]:
#import os
#os.listdir('../data/yelp_review_full/')

In [4]:
import os
import numpy as np
from ezpz import setup_torch
from hydra.utils import instantiate
from ngpt.configs import get_config, PROJECT_ROOT
from ngpt.trainer import Trainer
from enrich.console import get_console

console = get_console()
HF_DATASETS_CACHE = PROJECT_ROOT.joinpath('.cache', 'huggingface')
HF_DATASETS_CACHE.mkdir(exist_ok=True, parents=True)

os.environ['MASTER_PORT'] = '5278'
os.environ['HF_DATASETS_CACHE'] = HF_DATASETS_CACHE.as_posix()

SEED = np.random.randint(2**32)
console.log(f'SEED: {SEED}')

rank = setup_torch('DDP', seed=1234)
cfg = get_config(
    [
        'data=yelp',
        #'data.dataset=yelp_review_full',
        #'data.root_path=../data/yelp_review_full/',
        #'data.out_dir=output_yelp_review_full',
        'model=gpt2_medium',
        'optimizer=gpt2_medium',
        'train=gpt2_medium',
        'train.dtype=bfloat16',
        'train.max_iters=1000',
        'train.log_interval=100',
        'train.init_from=gpt2-medium',
    ]
)
config = instantiate(cfg)
trainer = Trainer(config)

--------------------------------------------------------------------------

  Local host:   thetagpu23
  Local device: mlx5_0
--------------------------------------------------------------------------


[2023-11-30 07:10:33][INFO][configs.py:72] - Setting HF_DATASETS_CACHE to /lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/.cache/huggingface/datasets
Failed to download font: Source Sans Pro, skipping!
Failed to download font: Titillium WebRoboto Condensed, skipping!


[07:10:36] SEED: 3164946669                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

[2023-11-30 07:10:37][INFO][configs.py:295] - Loading test from /lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/data/yelp_review_full/test.bin
[2023-11-30 07:10:37][INFO][configs.py:295] - Loading train from /lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/data/yelp_review_full/train.bin
[2023-11-30 07:10:37][WARNING][configs.py:330] - No meta.pkl found, assuming GPT-2 encodings...
[2023-11-30 07:10:38][INFO][configs.py:270] - Rescaling GAS -> GAS // WORLD_SIZE = 1 // 1
[2023-11-30 07:10:38][INFO][configs.py:432] - Tokens per iteration: 4,096
[2023-11-30 07:10:38][INFO][configs.py:454] - Using <torch.amp.autocast_mode.autocast object at 0x7f2cc6117fd0>
[2023-11-30 07:10:38][INFO][trainer.py:187] - Initializing from OpenAI GPT-2 Weights: gpt2-medium
[2023-11-30 07:10:45][INFO][model.py:225] - loading weights from pretrained gpt: gpt2-medium
[2023-11-30 07:10:45][INFO][model.py:234] - forcing vocab_size=50257, blo

## Prompt (**prior** to training)

In [5]:
query = "What is a supercomputer? Explain like I'm a child, and speak clearly. Double check your logic.."
outputs = trainer.evaluate(query, num_samples=1, display=False)
console.print(fr'\[prompt]: "{query}"')
console.print("\[response]:\n\n" + fr"{outputs['0']['raw']}")

[prompt]: "What is a supercomputer? Explain like I'm a child, and speak clearly. Double check your logic.."

[response]:

What is a supercomputer? Explain like I'm a child, and speak clearly. Double check your logic.. When I ask you to describe your computer, this is what you know: a computer that runs a quantum computer to run a computer program. There are now two supercomputers that can do quantum computations, so how can you explain quantum computing? We will get into what quantum computing is first, and then we are going to talk to you about what is a supercomputer. You will learn how to explain quantum computing to a child, and you will find out that quantum computing is what is called artificial intelligence (AI). Do you understand quantum computing?

Follow up – Answering the Quantum Computing Questions

Cognitive scientists are trying to define what is called artificial intelligence (AI). They are trying to understand what computers can do. At the heart of the argument are the following three questions:

What is AI and how does it apply to science?

What if I say, "A computer can tell me if I am an elf, a lion, a fox, or a squirrel?". Can I make a decision?

How does AI fit into the human condition?

Have you ever thought about what is intelligence? Have you ever wondered, "What if I want to create a computer that can build me an AI, and that computer is smarter than me? How will the AI affect my life?".

Before you even begin…

BECAUSE YOU HAVE NOT BEEN PURCHASED

This is where you may want to get an answer to a question that you have never had the opportunity to ask, and which is too big to ask right now. Think about it. Will I be able to make decisions? Will I be able to ask questions? Do I have the right to be an AI?

With an intelligent AI, you can have the ability to run my brain. You can have your mind, and you can control it. You can control your body, you can control your thoughts, and you can control your actions. That will make you a living being. If you are able to experience the world like the rest of us, you will have a better understanding of what it means to be alive; a living being. You will have the ability to observe, dream, and experience. All this will be possible to you if you are created as an intelligent being.

By creating an AI, this will be easier. It will be easy for someone to create an AI, and then run it as if

## Train Model


|  **NAME**  |     **DESCRIPTION**          |
|:----------:|:----------------------------:|
|   `step`   | Current training step        |
|   `loss`   | Loss value                   |
|   `dt`     | Time per step (in **ms**)    |
|   `sps`    | Samples per second           |
|   `mtps`   | (million) Tokens per sec     |
|   `mfu`    | Model Flops Utilization*     |
^Logging Legend

*in units of A100 `bfloat16` peak FLOPS

In [6]:
trainer.train()

  0%|          | 0/1000 [00:00<?, ?it/s]

[2023-11-30 07:12:55][INFO][trainer.py:518] - step=100 loss=3.118 dt=280.748 sps=3.562 mtps=0.015 mfu=34.014 train_loss=3.389 val_loss=0.000
[2023-11-30 07:13:23][INFO][trainer.py:518] - step=200 loss=3.138 dt=281.203 sps=3.556 mtps=0.015 mfu=34.008 train_loss=3.389 val_loss=0.000
[2023-11-30 07:13:51][INFO][trainer.py:518] - step=300 loss=3.065 dt=280.476 sps=3.565 mtps=0.015 mfu=34.012 train_loss=3.389 val_loss=0.000
[2023-11-30 07:14:19][INFO][trainer.py:518] - step=400 loss=3.194 dt=280.566 sps=3.564 mtps=0.015 mfu=34.015 train_loss=3.389 val_loss=0.000
[2023-11-30 07:14:48][INFO][trainer.py:518] - step=500 loss=3.086 dt=281.708 sps=3.550 mtps=0.015 mfu=34.003 train_loss=3.389 val_loss=0.000
[2023-11-30 07:15:16][INFO][trainer.py:518] - step=600 loss=3.055 dt=280.845 sps=3.561 mtps=0.015 mfu=34.003 train_loss=3.389 val_loss=0.000
[2023-11-30 07:15:44][INFO][trainer.py:518] - step=700 loss=3.213 dt=281.231 sps=3.556 mtps=0.015 mfu=33.998 train_loss=3.389 val_loss=0.000
[2023-11-30 0

## Evaluate Model

In [7]:
query = "What is a supercomputer? Explain like I'm a child, and speak clearly. Double check your logic.."
outputs = trainer.evaluate(query, num_samples=1, display=False)
console.print(fr'\[prompt]: "{query}"')
console.print("\[response]:\n\n" + fr"{outputs['0']['raw']}")

[prompt]: "What is a supercomputer? Explain like I'm a child, and speak clearly. Double check your logic.."

[response]:

What is a supercomputer? Explain like I'm a child, and speak clearly. Double check your logic..what's left to say and how do I want to be treated? I find a copy of my birthday song and I love it. What do you think is going to happen? I guess I got two chances on the table. \n\nWell, I like the mix of the place. Too cute for me to give a review.<|endoftext|>I don't know if I've already been to the MOST expensive hotel in town or not.  But, I like this hotel.  It's just a bit nicer than most others, like the Monte Carlo, but the rooms here are just fine. It isn't fancy, but it's not cheap for Vegas.<|endoftext|>I booked a room here for 2 nights. I stayed in the casino at 9pm on a Sunday.  I was charged for 1/5 of a soda instead of the full size. How underhanded! And I'm sure I'm very lucky to be charged for a soda which I had not paid for. I had paid for a soda, but never received it. I just want to make the situation clear:  whatever the hotel is doing to try to earn the best possible customer experience, they are doing a very poor job. \n\nTHOUGH THE RESOURCES ABOUT THIS PLACE ARE SO MUCH BETTER THAN ANY OF THE NEW YORK PLAZA PLAZA, THE CHARGES ARE EXCELLENT.\n\nI had a free birthday here with my room; however, I would never stay here again.<|endoftext|>I came here for a business trip and needed a room and price was about $100 per night. I was staying with my wife for a wedding. I had so much money I didn't even know what to expect. We got a room right away. It was amazing. The room was beautiful and clean. It was slightly busy but that's expected. There were 2 beds in the room. They were beautiful. The room was the best part. The bathroom was awesome. I'll go back to check it out!<|endoftext|>I stayed at the MOST expensive hotel in Vegas.  \n\nI didn't think it was bad but I was wrong.\n\nI talked to the front desk and they didn't ask me what room I was on or what hotel I was staying in.  I had booked two rooms on the same floor as the one I was on.  \n\nThe MOST expensive hotel